<a href="https://colab.research.google.com/github/bvu36/vuong598/blob/main/vuong598hw3p1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Bryan Vuong
# MAE 598 HW 3 Problem1


import numpy as np
import torch
from torch.autograd import Variable
import matplotlib.pyplot as plt
from IPython import display
import math 

# Variables and Givens
x1 = np.array([[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]])
x2 = np.flip(x1, axis=1).copy()
a = np.array(([[8.07131, 1730.63, 233.426], [7.43155, 1554.679, 240.337]]))
T = 20

# equations
pwater = 10 ** (a[0, 0] - a[0,1] / (T + a[0,2]))
pdiox = 10 ** (a[1, 0] - a[1,1] / (T + a[1,2]))
p = np.array([[28.1, 34.4, 36.7, 36.9, 36.8, 36.7, 36.5, 35.4, 32.9, 27.7, 17.5]])
p = torch.tensor(p, requires_grad=False, dtype=torch.float32)

A = Variable(torch.tensor([1.0, 1.0]), requires_grad=True)
x1torch = torch.tensor(x1, requires_grad=False, dtype=torch.float32)
x2torch = torch.tensor(x2, requires_grad=False, dtype=torch.float32)

a = 0.0001

# for loop for prediction
for i in range(100):
              ppredict = x1torch * torch.exp(A[0] * (A[1] *x2torch / (A[0] * x1torch + A[1] * x2torch)) ** 2) * pwater + \
                  x2torch * torch.exp(A[1] * (A[0] * x1torch / (A[0] * x1torch + A[1] * x2torch)) ** 2) *pdiox

              loss = (ppredict - p) ** 2
              loss = loss.sum()

              loss.backward()

              with torch.no_grad():
                A -= a * A.grad

                A.grad.zero_()

print('A12 and A21 are ',A)
print('The loss is ', loss.data.numpy())


# Comparison with HW1 -----------------------------------------------
import numpy as np
from scipy.optimize import minimize
 

# Equation and Definitions
def f(x):
  #variables
    x1 = x[0]
    x2 = x[1] 
    x3 = x[2] # A12
    x4 = x[3] # A21
    x5 = x[4] #psat
    return x1 * (2.7183 ** (x3 * ((x4 * x2) / ((x3 * x1) + (x4 * x2)) )** 2 )) * x5

def constraints1(x): 
    return x[0] + x[1] - 1


# initial guess
x0 = [0.5,0.5,3,5,36]


# bounds
bnds = ((0,1),(0,1),(-10,10),(-10,10),(17.5,36.9))

# constraints
cons = ({'type': 'eq', 'fun': constraints1},
)

# answers
ans = minimize(f,x0,method='SLSQP',bounds=bnds,constraints=cons)
x = ans.x

print()
print('Answer using HW1')
print('x1 = ' + str(x[0]))
print('x2 = ' + str(x[1]))
print('x3 = ' + str(x[2]))
print('x4 = ' + str(x[3]))




A12 and A21 are  tensor([1.9110, 1.7293], requires_grad=True)
The loss is  0.87836385

Answer using HW1
x1 = 0.0
x2 = 1.0
x3 = -7.123572804138453
x4 = -10.0
